In [119]:
import pandas as pd
from openpyxl import Workbook
from openpyxl import load_workbook
from google.cloud import translate
import google.auth
import itertools

### Load Excel Workbook

In [125]:
"""
check for repeated values - don't want to google the same value over and over

check list of column headers and translate

allow for list of columns to exclude/translate

check for uniqueness of columns to treat text and 'codes' separately


"""

fname = 'test'

In [115]:
wb = load_workbook(fname'.xlsx')
#print(wb.sheetnames)
ws = wb['Spanish']

### Library to store translated phrases ... to avoid uneccesary API calls

In [116]:
translated = {}

### Setup Google Translate API Client

In [121]:
# this notebook uses googles default credentials 
credentials, project = google.auth.default()

# api client to manage translate calls
translate_client = translate.Client(credentials=credentials)

In [122]:
def translate(phrase, from_lang='es', to_lang='en'):
    print "-"*40
    print "TRANSLATE: ", phrase
    print
    
    if phrase in translated:
        print "Already Translated"
        print translated[phrase]['translatedText']
        return translated[phrase]['translatedText']
        
    else:
        t = translate_client.translate(phrase, target_language=to_lang, format_='text')
        translated[phrase] = t
        print "New Translation"
        print t['translatedText']
        return t['translatedText']

### Iterate Worksheet and Translate Cells

In [123]:
for col in ws.iter_cols():
    for cell in col:
        
        if cell.col_idx == 2:
            #cell.value = 'new - ' + cell.value.encode('utf-8')
            cell.value = translate(cell.value)
        
        
        #print(cell.value, cell.anchor, cell.row, cell.col_idx)

----------------------------------------
TRANSLATE:  Spanish

New Translation
Spanish
----------------------------------------
TRANSLATE:  Buenos días.

New Translation
Good Morning.
----------------------------------------
TRANSLATE:  Buenas tardes.

New Translation
Good afternoon.
----------------------------------------
TRANSLATE:  Buenas noches.

New Translation
Goodnight.
----------------------------------------
TRANSLATE:  Hola, me llamo Juan.

New Translation
Hello, my name is Juan.
----------------------------------------
TRANSLATE:  ¿Cómo se llama usted?

New Translation
What is your name?
----------------------------------------
TRANSLATE:  ¿Cómo está usted?

New Translation
How are you?
----------------------------------------
TRANSLATE:  Estoy bien.

New Translation
I'm fine.
----------------------------------------
TRANSLATE:  Mucho gusto.

New Translation
Pleasure.
----------------------------------------
TRANSLATE:  Adiós.

New Translation
Bye.
--------------------------

In [124]:
wb.save(fname+'_translated.xlsx')

### For later ... work with data in pandas

In [6]:
# if sheet has headers and NO index

data = ws.values
cols = next(data)
data = list(data)
data = (itertools.islice(r, 0, None) for r in data)
df = pd.DataFrame(data, columns=cols)

"""
# if sheet has headers and index

data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (itertools.islice(r, 1, None) for r in data)
df = pd.DataFrame(data, index=idx, columns=cols)

# if sheet is just data

df = pd.DataFrame(ws.values)
"""

'\n# if sheet has headers and index\n\ndata = ws.values\ncols = next(data)[1:]\ndata = list(data)\nidx = [r[0] for r in data]\ndata = (itertools.islice(r, 1, None) for r in data)\ndf = pd.DataFrame(data, index=idx, columns=cols)\n\n# if sheet is just data\n\ndf = pd.DataFrame(ws.values)\n'

In [7]:
df

,English,Spanish,Pronunciation
0,Good morning.,Buenos días.,booEHN-os DEE-as
1,Good afternoon.,Buenas tardes.,booEHN-as TAR-dehs
2,Good evening. (greeting),Buenas noches.,booEHN-as NO-chehs
3,"Hello, my name is John.","Hola, me llamo Juan.",OH-la meh YA-mo Wahn
4,What is your name?,¿Cómo se llama usted?,KOH-moh seh YA-mah oos-TEHD
5,How are you?,¿Cómo está usted?,KOH-moh ehs-TA oos-TEHD
6,I am fine.,Estoy bien.,ehs-TOY bee-EHN
7,Nice to meet you.,Mucho gusto.,MOO-choh GOOS-toh
8,Goodbye.,Adiós.,ah-dee-OHS
9,See you later.,Hasta luego.,AHS-ta looEH-go


In [8]:
# check for numeric or date columns and insert text cols into dict

text = {}

for name, col in df.iteritems():
    
    try:
        pd.to_numeric(col)
        
    except:
        
        try:
            pd.to_date(col)
            
        except:
            text[name] = col

In [9]:
print text

{u'Pronunciation': 0                                      booEHN-os DEE-as
1                                    booEHN-as TAR-dehs
2                                    booEHN-as NO-chehs
3                                  OH-la meh YA-mo Wahn
4                           KOH-moh seh YA-mah oos-TEHD
5                               KOH-moh ehs-TA oos-TEHD
6                                       ehs-TOY bee-EHN
7                                     MOO-choh GOOS-toh
8                                            ah-dee-OHS
9                                       AHS-ta looEH-go
10    ehs-TOY pehr-DEE-doh. DOHN-deh ehs-TA el BAH-neeo
11                kohn pehr-MEE-soh OR pehr-DOH-nah-meh
12                                         pohr fah-VOR
13                                          gra-SEE-ahs
14                                       low see-EHN-to
15                                             sah-LOOD
16                                            deh NA-da
17                           

In [17]:
translate_client.translate("hola mi amigo gorda", target_language='en')

{u'detectedSourceLanguage': u'es',
 'input': 'hola mi amigo gorda',
 u'translatedText': u'Hello my fat friend'}

In [93]:
s.split(' ')

['once', '**&', 'upon', 'a', '..#', 'time', 'in', '99327', 'lando']

In [14]:
s = "once **& upon a ..# time in 99327 lando"

In [ ]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False